In [75]:
import ee
import numpy as np
import cmocean.cm
import matplotlib.colors

def cm2ee(cm, num=256):
    hexs = [
        matplotlib.colors.rgb2hex(rgb)[1:] 
        for rgb 
        in cm(np.linspace(0, 1, num=num))
    ]
    return ",".join(hexs)


In [19]:
ee.Initialize()


In [58]:
ssh = ee.ImageCollection("users/fbaart/ssh_grids_v1609")
ssh_trend = ee.Image('users/fbaart/ssh-trend')

first = ssh.first()
# Global 6 point polygon, avoids incorrect wrapping
global_polygon = ee.Geometry.Polygon(
  [
    [
      [-180, 90],
      [0, 90],
      [180, 90],
      [180, -90],
      [0, -90],
      [-180, -90]
    ]
  ], 
  None, 
  False
)
# Lookup the scale
scale = first.projection().nominalScale()

In [53]:
# Setup independent variables

def add_independent_variables(image):
    """setup independent variables and add them as bands to an image"""
    constant = ee.Image.constant(1).rename('constant')
    t = image.date().millis()
    # ms -> s -> m -> hr -> day -> year
    years = (ee.Image.constant(t)
        .divide(1e3 * 60 * 60 * 24 * 365.25)
        .float()
        .rename('time'))
    # seasonal cycle
    time_radians = (ee.Image.constant(t)
        .divide(1e3 * 60 * 60 * 24 * 365.25)
        .multiply(2 * np.pi)
        .float()
        .rename('time-radians'))
    # nodal cycle
    nodal_radians = (ee.Image.constant(t)
        .float()
        .divide(1e3 * 60 * 60 * 24 * 365.25 * 18.613)
        .multiply(2 * np.pi)
        .float()
        .rename('nodal-radians'))
    # linearize
    cos_time = time_radians.cos().rename('cos')
    sin_time = time_radians.sin().rename('sin')
    cos_nodal_time = nodal_radians.cos().rename('cos-nodal')
    sin_nodal_time = nodal_radians.sin().rename('sin-nodal')
    result = (image
        .addBands(years)
        .addBands(cos_time)
        .addBands(sin_time)
        .addBands(cos_nodal_time)
        .addBands(sin_nodal_time)
        .addBands(constant)
        .addBands(time_radians)
        .addBands(nodal_radians))
    return result



In [54]:
# define the regression function that selects data, extracts and flattens the coefficients
def regression(collection, dependent, independent):
    all_vars = independents + dependent
    fit = (
        collection.select(all_vars)
            .reduce(ee.Reducer.linearRegression(len(independents), 1))
            .select('coefficients')
            .arrayProject([0])
            .arrayFlatten([independents])
    )
    return fit
    

In [55]:
def map2id(image, vis_params):
    m = image.getMapId(vis_params)
    mapid = m.get('mapid')
    token = m.get('token')

    url = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'.format(
      mapid=mapid,
      token=token
    )
    result = {
      'mapid': mapid,
      'token': token,
      'url': url,
      'vis_params': vis_params
    }
    return result
def map2url(image, vis_params):
    obj = map2id(image, vis_params)
    return obj['url']
    



In [76]:
# TODO allow to filter by time
equations = ssh.map(add_independent_variables)

independents = ['constant', 'time']
dependent = ['b1']
fit = regression(equations, dependent, independents)

vis_params = {
    'min': -10,
    'max': 10,
    'palette': cm2ee(cmocean.cm.balance)
}
url = map2url(fit.select('time'), vis_params)        



In [96]:

max = trend.reduceRegion(ee.Reducer.max(), global_polygon).getInfo()['time']
min = trend.reduceRegion(ee.Reducer.min(), global_polygon).getInfo()['time']
domain = np.max(np.abs([max, min]))
vis_params = {
    'min': -domain*0.3,
    'max': domain*0.3,
    'palette': cm2ee(cmocean.cm.balance)
}
trend = ssh_trend.select('time')
obj = map2id(trend, vis_params=vis_params)
obj

{'mapid': '99ebdb03dc6c7fa1ad655dd6783cb0eb',
 'token': '20bee80feb977f3e4892d2f036868d3f',
 'url': 'https://earthengine.googleapis.com/map/99ebdb03dc6c7fa1ad655dd6783cb0eb/{z}/{x}/{y}?token=20bee80feb977f3e4892d2f036868d3f'}

0.11486345529556274